In [1]:
import os

In [2]:
%pwd

'e:\\Project\\DS Project\\E to E\\ML\\Credit Utility\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Project\\DS Project\\E to E\\ML\\Credit Utility'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Neeraj2210/Credit_Utility.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Neeraj2210"
os.environ["MLFLOW_TRACKING_PASSWORD"]="971396da3b9c80d0dd96cf17720b35c7098df00e"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from CREDIT_UTILITY.constants import *
from src.CREDIT_UTILITY.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.xgboost
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])


        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Neeraj2210/Credit_Utility.mlflow",
           
        )

        return model_evaluation_config


In [12]:
import os
import sys
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
import joblib
from urllib.parse import urlparse
from CREDIT_UTILITY.logger import logger
from CREDIT_UTILITY.Exception import CustomException

In [13]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        return accuracy, precision, recall, f1
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            (accuracy, precision, recall, f1) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local

            scores = {"accuracy": accuracy, "precision": precision, "recall": recall, 'f1':f1}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="xgboost")
            else:
                mlflow.sklearn.log_model(model, "model")
            

In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()

except Exception as e:
    error = CustomException(e, sys)
    logger.info(error.error_message)

[2024-03-04 07:59:42,499: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-04 07:59:42,512: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-04 07:59:42,527: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-04 07:59:42,535: INFO: common: created directory at: artifacts]
[2024-03-04 07:59:42,535: INFO: common: created directory at: artifacts/model_evaluation]


[2024-03-04 07:59:44,734: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


e:\Project\DS Project\E to E\ML\Credit Utility\utility_env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'xgboost' already exists. Creating a new version of this model...
2024/03/04 08:00:01 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: xgboost, version 3
Created version '3' of model 'xgboost'.
